### Импортируем библиотеки

In [18]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import numpy as np

from sklearn.metrics import accuracy_score, precision_recall_curve,precision_score, plot_roc_curve
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

## Проводим Data Understanding данных файлов ga_hits и ga_session

### Работаем с данными из файла ga_hits

In [19]:
df = pd.read_csv('ga_hits.csv') #загружаем датасет и присваем ему переменную
df.head(10)

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN
5,6466333295973247896.1640246168.1640246168,2021-12-23,704194.0,41,event,NaN,sberauto.com/cars?utm_source_initial=youtube&u...,quiz,quiz_show,NaN,NaN
6,281215510786615563.1640245516.1640245516,2021-12-23,911854.0,102,event,NaN,sberauto.com/cars/toyota?isnew=false&rental_ca...,quiz,quiz_show,NaN,NaN
7,4024492994895054107.1640269084.1640269084,2021-12-23,1071804.0,85,event,NaN,sberauto.com/cars/all/mercedes-benz/glc/f8f330...,quiz,quiz_show,NaN,NaN
8,555009234841130092.1640256620.1640256620,2021-12-23,1013899.0,101,event,VloVXNWduHeTjUoDkjkO,sberauto.com/cars/all/kia/sorento/c38179cb?utm...,quiz,quiz_show,NaN,NaN
9,2692901778487480807.1640206845.1640206845,2021-12-23,0.0,1,event,NaN,sberauto.com/cars/all/nissan/x-trail/0744675f?...,card_web,view_card,NaN,NaN


In [3]:
df.info() #смотрим какой тип данных в стобцах

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15726470 entries, 0 to 15726469
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      object 
 1   hit_date        object 
 2   hit_time        float64
 3   hit_number      int64  
 4   hit_type        object 
 5   hit_referer     object 
 6   hit_page_path   object 
 7   event_category  object 
 8   event_action    object 
 9   event_label     object 
 10  event_value     float64
dtypes: float64(2), int64(1), object(8)
memory usage: 1.3+ GB


In [4]:
df.shape #проверяем замерность датасета

(15726470, 11)

In [5]:
df.describe(include='all') #выводим описательные статистики всех данных по столбцам

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
count,15726470,15726470,6.566148e+06,1.572647e+07,15726470,9451666,15726470,15726470,15726470,11966286,0.0
unique,1734610,226,NaN,NaN,1,37873,342715,52,230,39825,NaN
top,5442565791571325612.1632449195.1632449195,2021-05-24,NaN,NaN,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/,card_web,view_card,KclpemfoHstknWHFiLit,NaN
freq,768,513035,NaN,NaN,15726470,8879187,2793639,7456998,3558985,6505447,NaN
mean,NaN,NaN,2.091050e+05,2.356715e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,4.032110e+05,2.887713e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2.412000e+04,7.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,8.994100e+04,1.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.195352e+05,2.900000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Работаем с данными из файла ga_sessions

In [20]:
df1 = pd.read_csv('skillbox_diploma_main_dataset_sberautopodpiska-20230601T122423Z-003/skillbox_diploma_main_dataset_sberautopodpiska/ga_sessions.csv')
df1.head(10)

C:\Users\Полина\AppData\Local\Temp\ipykernel_9132\3000612195.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('skillbox_diploma_main_dataset_sberautopodpiska-20230601T122423Z-003/skillbox_diploma_main_dataset_sberautopodpiska/ga_sessions.csv')


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.163776,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.163687,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.164065,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.162225,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.162225,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
5,9055447192389856083.1622453074.1622453074,2108385598.162245,2021-05-31,12:00:00,1,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Apple,NaN,375x812,Safari,Russia,Saint Petersburg
6,9055455318486370642.1640843788.1640843788,2108387490.164084,2021-12-30,08:56:28,1,TxKUcPpthBDPieTGmVhx,cpc,FTjNLDyTrXaWYgZymFkV,LcGIUNPUAmXtQJaDfFBR,PwscUHjoUJDrtfWESIHj,tablet,Android,Lenovo,NaN,602x1029,YaBrowser,Russia,Saint Petersburg
7,9055461992850812764.1626107740.1626107740,2108389044.162611,2021-07-12,19:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Samsung,NaN,360x640,Chrome,Russia,Saint Petersburg
8,9055462349345527315.1638536723.1638536723,2108389127.163854,2021-12-03,16:05:23,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,390x844,Safari,Russia,Moscow
9,9055466554104774132.1624800757.1624800757,2108390106.16248,2021-06-27,16:00:00,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Samsung,NaN,412x915,Chrome,Russia,Moscow


In [21]:
df_merged = pd.merge(
    df, df1,
    left_on='session_id',
    right_on='session_id',
    how='inner'
) #объединям два датафрейма и запысываем в переменную

df_merged

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,...,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
1,5639623078712724064.1640254056.1640254056,2021-12-23,290095.0,12,event,NaN,sberauto.com/cars/all/kia/seltos/20f30855?utm_...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
2,5639623078712724064.1640254056.1640254056,2021-12-23,294857.0,18,event,NaN,sberauto.com/cars/all/volkswagen/tiguan/0208cd...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
3,5639623078712724064.1640254056.1640254056,2021-12-23,295591.0,20,event,NaN,sberauto.com/cars/all/volkswagen/tiguan/0208cd...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
4,5639623078712724064.1640254056.1640254056,2021-12-23,290039.0,8,event,NaN,sberauto.com/cars/all/kia/seltos/20f30855?utm_...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15685214,1706097853564481669.1640267190.1640267190,2021-12-23,0.0,1,event,NaN,sberauto.com/moskva/cars?datefrom=2021&rental_...,quiz,quiz_show,NaN,...,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,428x926,Safari,Russia,Moscow
15685215,8262758806963127884.1640272536.1640272536,2021-12-23,0.0,1,event,NaN,sberauto.com/cars/all/renault/logan/8c3c73f2?u...,quiz,quiz_show,NaN,...,DZYjhfIUfdqhfuTNUmjn,puhZPIYqKXeFPaUviSjo,mobile,Android,Realme,NaN,360x800,Chrome,Russia,Ulyanovsk
15685216,3349670872968620291.1640264206.1640264206,2021-12-23,0.0,1,event,NaN,sberauto.com/cars/all/kia/rio/fee33fe6?rental_...,quiz,quiz_show,NaN,...,NaN,NaN,mobile,iOS,Apple,NaN,375x667,Safari (in-app),Russia,Samara
15685217,1009509786569589790.1640244938.1640244938,2021-12-23,600274.0,3,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,...,NaN,tVtbIKrPSOvrXLCznVVe,desktop,Windows,NaN,NaN,1366x768,Edge,Russia,Orenburg


In [22]:
df_1 = df_merged.iloc[0:3137043]
df_2 = df_merged.iloc[3137044:6274086]
df_3 = df_merged.iloc[6274087:9411129]
df_4 = df_merged.iloc[9411130:12548172]
df_5 = df_merged.iloc[12548173:15685219]

In [23]:
df_1

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,...,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
1,5639623078712724064.1640254056.1640254056,2021-12-23,290095.0,12,event,NaN,sberauto.com/cars/all/kia/seltos/20f30855?utm_...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
2,5639623078712724064.1640254056.1640254056,2021-12-23,294857.0,18,event,NaN,sberauto.com/cars/all/volkswagen/tiguan/0208cd...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
3,5639623078712724064.1640254056.1640254056,2021-12-23,295591.0,20,event,NaN,sberauto.com/cars/all/volkswagen/tiguan/0208cd...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
4,5639623078712724064.1640254056.1640254056,2021-12-23,290039.0,8,event,NaN,sberauto.com/cars/all/kia/seltos/20f30855?utm_...,card_web,view_new_card,NaN,...,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137038,7498262934740319036.1639118594.1639118594,2021-12-10,14339.0,12,event,NaN,sberauto.com/cars/all/lada-vaz/vesta/2fc745ed?...,card_web,view_card,NaN,...,NaN,ITfrGJNwVsHBGJMAKoct,mobile,Android,Vivo,NaN,393x876,YaBrowser,Russia,Saint Petersburg
3137039,7498262934740319036.1639118594.1639118594,2021-12-10,41874.0,21,event,NaN,sberauto.com/cars/all/lada-vaz/vesta/2fc745ed?...,card_web,view_new_card,NaN,...,NaN,ITfrGJNwVsHBGJMAKoct,mobile,Android,Vivo,NaN,393x876,YaBrowser,Russia,Saint Petersburg
3137040,7498262934740319036.1639118594.1639118594,2021-12-10,41870.0,20,event,NaN,sberauto.com/cars/all/lada-vaz/vesta/2fc745ed?...,card_web,view_card,NaN,...,NaN,ITfrGJNwVsHBGJMAKoct,mobile,Android,Vivo,NaN,393x876,YaBrowser,Russia,Saint Petersburg
3137041,7498262934740319036.1639118594.1639118594,2021-12-10,43663.0,24,event,NaN,sberauto.com/cars/all/lada-vaz/vesta/2fc745ed?...,card_web,view_card,NaN,...,NaN,ITfrGJNwVsHBGJMAKoct,mobile,Android,Vivo,NaN,393x876,YaBrowser,Russia,Saint Petersburg


In [24]:
df_1.to_csv('~/downloads/df_1', index=False)

In [8]:
df_merged.info() #смотрим какой тип данных в стобцах

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15685219 entries, 0 to 15685218
Data columns (total 28 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   session_id                object 
 1   hit_date                  object 
 2   hit_time                  float64
 3   hit_number                int64  
 4   hit_type                  object 
 5   hit_referer               object 
 6   hit_page_path             object 
 7   event_category            object 
 8   event_action              object 
 9   event_label               object 
 10  event_value               float64
 11  client_id                 object 
 12  visit_date                object 
 13  visit_time                object 
 14  visit_number              int64  
 15  utm_source                object 
 16  utm_medium                object 
 17  utm_campaign              object 
 18  utm_adcontent             object 
 19  utm_keyword               object 
 20  device_category       

In [9]:
df_merged.shape #проверяем замерность датасета

(15685219, 28)

In [10]:
df_merged.describe(include='all') #выводим описательные статистики всех данных по столбцам

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,...,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
count,15685219,15685219,6.525016e+06,1.568522e+07,15685219,9449721,15685219,15685219,15685219,11957537,...,12852801,6480392,15685219,6526837,11739344,122287,15685219,15685219,15685219,15685219
unique,1732266,226,NaN,NaN,1,37826,342242,52,230,39777,...,280,1192,3,13,200,104,4947,55,159,2389
top,5442565791571325612.1632449195.1632449195,2021-05-24,NaN,NaN,event,HbolMJUevblAbkHClEQa,podpiska.sberauto.com/,card_web,view_card,KclpemfoHstknWHFiLit,...,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Apple,AuMdmADEIoPXiWpTsBEj,414x896,Chrome,Russia,Moscow
freq,768,513035,NaN,NaN,15685219,8879128,2790578,7431798,3546622,6505347,...,9647665,4717082,11595541,3433515,4708652,78321,1480233,8197216,15270908,6795118
mean,NaN,NaN,2.042751e+05,2.347792e+01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,3.785350e+05,2.861302e+01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,0.000000e+00,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,2.416700e+04,7.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,8.973300e+04,1.500000e+01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.181740e+05,2.900000e+01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_merged_name_columns = df_merged.columns.values.tolist() #выводим все названия столбцов df
df_merged_name_columns

['session_id',
 'hit_date',
 'hit_time',
 'hit_number',
 'hit_type',
 'hit_referer',
 'hit_page_path',
 'event_category',
 'event_action',
 'event_label',
 'event_value',
 'client_id',
 'visit_date',
 'visit_time',
 'visit_number',
 'utm_source',
 'utm_medium',
 'utm_campaign',
 'utm_adcontent',
 'utm_keyword',
 'device_category',
 'device_os',
 'device_brand',
 'device_model',
 'device_screen_resolution',
 'device_browser',
 'geo_country',
 'geo_city']

### Определение типов переменных в датасете df_merged
 'session_id',
 'hit_date',
 'hit_time',
 'hit_number',
 'hit_type',
 'hit_referer',
 'hit_page_path',
 'event_category',
 'event_action',
 'event_label',
 'event_value',
 'client_id',
 'visit_date',
 'visit_time',
 'visit_number',
 'utm_source',
 'utm_medium',
 'utm_campaign',
 'utm_adcontent',
 'utm_keyword',
 'device_category',
 'device_os',
 'device_brand',
 'device_model',
 'device_screen_resolution',
 'device_browser',
 'geo_country',
 'geo_city'


## Определение целевой переменной в датасете df_merged и создание новой фичи target

Целевая переменная: event_action Конкретные значения, которые должна принимать целевая переменная перечислены в PDF файле к проекту. Эти значения будут считаться за 1. Остальные значения за 0.

In [5]:
df_merged_copy = df_merged.copy() #копируем датасет для дальнейшего удаления выбросов и аномалий

In [14]:
distribution_percentage_df_merged.info()

<class 'pandas.core.series.Series'>
Index: 230 entries, view_card to click_on_android
Series name: event_action
Non-Null Count  Dtype  
--------------  -----  
230 non-null    float64
dtypes: float64(1)
memory usage: 11.7+ KB


In [6]:
event_action_list = ['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']

In [7]:
df_merged_copy['target'] = None

In [24]:
distribution_percentage_df_merged[['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']]

sub_car_claim_click                 0.237593
sub_car_claim_submit_click          0.077557
sub_open_dialog_click               0.162522
sub_custom_question_submit_click    0.003870
sub_call_number_click               0.023194
sub_callback_submit_click           0.018801
sub_submit_success                  0.114382
sub_car_request_submit_click        0.017743
Name: event_action, dtype: float64

In [25]:
percentage_of_target_values_df_merged = sum(distribution_percentage_df_merged[['sub_car_claim_click', 'sub_car_claim_submit_click',
'sub_open_dialog_click', 'sub_custom_question_submit_click',
'sub_call_number_click', 'sub_callback_submit_click', 'sub_submit_success',
'sub_car_request_submit_click']]) #узнаём процент совершенных целевых действий из всех действий в столбце

percentage_of_target_values_df_merged

0.6556618686675653

In [8]:
df_merged_copy['target'] = df_merged_copy.apply(lambda x: 1 if x.event_action in event_action_list else 0, axis=1)

In [13]:
df_merged_copy.target.value_counts()

0    15582377
1      102842
Name: target, dtype: int64

In [9]:
# Удаляем ненужные/лишние колонки 

columns_to_drop = [
    'session_id',
    'hit_date',
    'hit_time',
    'hit_number',
    'hit_type',
    'hit_referer',
    'hit_page_path',
    'event_category',
    'event_action',
    'event_label',
    'event_value',
    'client_id',
    'visit_date',
    'visit_time',
    'visit_number'
]

df_merged_copy.drop(columns_to_drop, axis=1, inplace=True) 

In [12]:
df_merged_copy

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,target
0,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg,0
1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg,0
2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg,0
3,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg,0
4,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,NaN,360x780,Chrome,Russia,Saint Petersburg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15685214,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,NaN,428x926,Safari,Russia,Moscow,0
15685215,IZEXUFLARCUMynmHNBGo,blogger_channel,foFTSdUvNqqkPzZvgiqt,DZYjhfIUfdqhfuTNUmjn,puhZPIYqKXeFPaUviSjo,mobile,Android,Realme,NaN,360x800,Chrome,Russia,Ulyanovsk,0
15685216,QxAxdyPLuQMEcrdZWdWb,cpa,mSVMBvlHFgftJaQAZEdp,NaN,NaN,mobile,iOS,Apple,NaN,375x667,Safari (in-app),Russia,Samara,0
15685217,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,tVtbIKrPSOvrXLCznVVe,desktop,Windows,NaN,NaN,1366x768,Edge,Russia,Orenburg,0


In [10]:
df_merged_copy_name_columns = df_merged_copy.columns.values.tolist() #выводим все названия столбцов df
df_merged_copy_name_columns

['utm_source',
 'utm_medium',
 'utm_campaign',
 'utm_adcontent',
 'utm_keyword',
 'device_category',
 'device_os',
 'device_brand',
 'device_model',
 'device_screen_resolution',
 'device_browser',
 'geo_country',
 'geo_city',
 'target']

In [14]:
for x in df_merged_copy_name_columns: #выводим все уникальные значения по столбцам
    print(f"Характеристика: {x} \nКоличество уникальных значений: {df_merged_copy[x].nunique()} \nСписок значений: \n{df_merged_copy[x].value_counts()} \n")

Характеристика: utm_source 
Количество уникальных значений: 280 
Список значений: 
ZpYIoDJMcFzVoPFsHGJL    5151702
fDLlAcSmythWSCVMvqvL    3250926
kjsLglQLzykiRbcDiGcD    1772431
bByPQxmDaMXgpHeypKSM    1341803
BHcvLfOaCWvWTykYqHVe    1231071
                         ...   
OwOCmIeWDzEyCQjXdfQO          1
VdeFdoGCqZBxFfKSHNLl          1
tYfqTBjAxWbmBruissGm          1
DBLWVjTjVZuzWYWTaDfZ          1
gSwaXkJgHiWoIntUiupK          1
Name: utm_source, Length: 280, dtype: int64 

Характеристика: utm_medium 
Количество уникальных значений: 55 
Список значений: 
banner               4898563
cpc                  3300503
(none)               3250926
referral             1920459
cpm                   738765
organic               646182
push                  252864
email                 241084
stories                95604
blogger_channel        87087
smartbanner            49146
tg                     33822
blogger_stories        33068
cpa                    31586
cpv                    27395
po

Характеристика: geo_city 
Количество уникальных значений: 2389 
Список значений: 
Moscow              6795118
Saint Petersburg    2365520
(not set)            596279
Yekaterinburg        328301
Krasnodar            293501
                     ...   
Kushimoto                 1
Eregli                    1
Eumseong-gun              1
Salsk                     1
Pa Tong                   1
Name: geo_city, Length: 2389, dtype: int64 

Характеристика: target 
Количество уникальных значений: 2 
Список значений: 
0    15582377
1      102842
Name: target, dtype: int64 



In [11]:
df_merged_copy.to_csv('~/downloads/df_merged_ready', index=False)

## Data Preparation

### Удаляем выбросы и анамолии в данных, обрабатываем пропуски.

In [25]:
df_merged_ready = pd.read_csv('~/downloads/df_merged_ready')

C:\Users\Полина\AppData\Local\Temp\ipykernel_704\221618566.py:1: DtypeWarning: Columns (4,6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged_ready = pd.read_csv('~/downloads/df_merged_ready')


In [17]:
def print_useful_rows_info(df_merged_ready): #создаём функцию для отоброжения полностью заполненых объектов выборки
    print('Количество полностью заполненных объектов из всей выборки:', 
          len(df_merged_ready.dropna()))
    print('Процент полностью заполненных объектов из всей выборки:', 
          round(len(df_merged_ready.dropna()) / len(df_merged_ready) * 100, 2))
    

print_useful_rows_info(df_merged_ready)

Количество полностью заполненных объектов из всей выборки: 110483
Процент полностью заполненных объектов из всей выборки: 0.7


In [15]:
missing_values_df_merged_ready = ((df_merged_ready.isna().sum() / len(df_merged_ready)) * 100).sort_values(ascending=False) #формула которая отоброжает процент пропущенных значений

print('Процент пропущенных значений:')
missing_values_df_merged_ready

Процент пропущенных значений:


device_model                99.220368
utm_keyword                 58.684721
device_os                   58.388614
device_brand                25.156646
utm_adcontent               18.057880
utm_campaign                14.018759
utm_source                   0.004463
utm_medium                   0.000000
device_category              0.000000
device_screen_resolution     0.000000
device_browser               0.000000
geo_country                  0.000000
geo_city                     0.000000
target                       0.000000
dtype: float64

In [16]:
missing_columns_df_merged_ready = missing_values_df_merged_ready[missing_values_df_merged_ready.values > 0].index.tolist() #присваеваем переменную списку всех колонок с пропусками
missing_columns_df_merged_ready

['device_model',
 'utm_keyword',
 'device_os',
 'device_brand',
 'utm_adcontent',
 'utm_campaign',
 'utm_source']

In [20]:
df_merged_ready.utm_adcontent.value_counts(dropna= False) #выводим список уникальных значений столбца и их количество, включая пропуски.

JNHcPlZPxEMWDnRiyoBf    9647665
NaN                     2832418
vCIpmpaGBnIQhyYNkXqp    1806637
xhoenQgDQsgfEPYNPwKO     345071
PkybGvWbaqORmxjNunqZ     199457
                         ...   
gQEnaeQmOHisHSTEyzhp          1
FsyvyvqcdOnQCoOklwfT          1
FvtsWdHfGtOlguWeXGof          1
fiImkyBmzQSeAeQEVCDZ          1
VlbHDutfcfYJARFNICFX          1
Name: utm_adcontent, Length: 281, dtype: int64

In [21]:
df_device_brand = df_merged_ready.device_brand.value_counts(dropna= False)
df_device_brand.head(10)

Apple        4708652
NaN          3945875
Samsung      2609769
Xiaomi       2071248
Huawei       1370025
Realme        142883
(not set)     132775
OnePlus        92520
OPPO           91988
Vivo           84768
Name: device_brand, dtype: int64

In [22]:
df_merged_ready.device_brand.unique()

array(['Huawei', 'Xiaomi', nan, 'Samsung', 'Apple', 'Google', 'Realme',
       'Motorola', '(not set)', 'Vivo', 'TCL', 'ZTE', 'OPPO', 'OnePlus',
       'POCO', 'DOOGEE', 'Tecno', 'Sony', 'Asus', 'Blackview', 'Infinix',
       'Oukitel', 'BQ', 'MXQ', 'Alcatel', 'Nokia', 'Meizu', 'Vsmart',
       'Lenovo', 'TP-Link', 'itel', 'LeTV', 'LG', 'Cubot', 'Umidigi',
       'Leagoo', 'HTC', 'Coolpad', 'Karbonn', 'BlackBerry', 'Artel',
       'Wileyfox', 'Flexymove', 'LeEco', 'CAT', 'Ulefone', 'Prestigio',
       'Neffos', 'Inoi', 'Sharp', 'HOMTOM', 'Micromax', 'Amazon', 'Nomu',
       'AGM', 'DEXP', 'InnJoo', 'Jiake', 'Philips', 'Acer', 'Kingplay',
       'Evercoss', 'Vertex', 'iNew', 'Wiko', 'Haier', 'Nuu', 'Hisense',
       'Smartisan', 'Alldocube', 'Fly', 'Advan', 'Digma', 'Black Fox',
       'Tesla', 'Kyocera', 'Keecoo', 'Billion', 'Mito', 'Egreat',
       'Mozilla', 'General Mobile', 'Celkon', 'Teclast', 'E&L', 'Gome',
       'BLUBOO', 'Highscreen', 'Corn', 'Wigor', 'Komu', 'Panasonic',
    

In [23]:
df_merged_ready.utm_campaign.value_counts(dropna= False)

LTuZkdKfxRGVceoWkVyg    5166521
LEoPHuyFvzoNfnzGgfcd    3177265
NaN                     2198873
gecBYcKZCPMcVYdSSzKP    1138547
FTjNLDyTrXaWYgZymFkV     750959
                         ...   
tWBQlsvNfHxRUjaPAfhd          1
cXxuwXPoQCvAXPHpFcZl          1
NUYvxgBEIYqMlnpeDTIM          1
ehLonfPENrOEoPTIyiOZ          1
XzfzEBYZWgSDtJNXOadn          1
Name: utm_campaign, Length: 407, dtype: int64

In [25]:
df_merged_ready.utm_source.value_counts(dropna= False)

ZpYIoDJMcFzVoPFsHGJL    5151702
fDLlAcSmythWSCVMvqvL    3250926
kjsLglQLzykiRbcDiGcD    1772431
bByPQxmDaMXgpHeypKSM    1341803
BHcvLfOaCWvWTykYqHVe    1231071
                         ...   
zPJpddwzkFqLMSYgtDqy          1
oBixMNCffOQtQcLCZfRN          1
OboZzsWwJIeGPLeiLGMq          1
LBaWJEeycBIqEqUqkoqw          1
gSwaXkJgHiWoIntUiupK          1
Name: utm_source, Length: 281, dtype: int64

In [26]:
df_merged_ready.utm_keyword.value_counts(dropna= False)

NaN                     9204827
puhZPIYqKXeFPaUviSjo    4717082
aXQzDWsJuGXeBXexNHjc     176453
ITfrGJNwVsHBGJMAKoct     138328
UKNBOHebRIIsQRsjNKay     132629
                         ...   
AWRuffbiANmJrYMwpzNm          1
gVRrcxiDQubJiljoTbGm          1
vpQKKTAzuMqZqbIphRKd          1
LbYgXRsCfbfDfXBVePCv          1
qFcyispArlKUQmFENgzo          1
Name: utm_keyword, Length: 1193, dtype: int64

In [28]:
df_merged_ready.device_model.value_counts(dropna= False)

NaN                     15562932
AuMdmADEIoPXiWpTsBEj       78321
tWBQlsvNfHxRUjaPAfhd        4339
pTgAEPipQxDXCjPrJbHo        3495
cwMJxNXiWUgMUxGiCTPs        3012
                          ...   
aWSBcBZlKorTFjJGlWKG           1
qmRODeCJLlmkmwxNYXvp           1
hOhIHYHnDhUlkzDoZJdA           1
UvyzWsKbojrmlVrFRIgp           1
mhkBNBhsTwAvDUZdstnC           1
Name: device_model, Length: 105, dtype: int64

In [29]:
df_merged_ready.device_os.value_counts(dropna= False)

NaN              9158382
Android          3433515
iOS              1735956
Windows          1015524
Macintosh         309734
Linux              30932
(not set)            448
Chrome OS            360
BlackBerry           211
Tizen                 99
Firefox OS            37
Nokia                 12
Windows Phone          7
Samsung                2
Name: device_os, dtype: int64

In [36]:
device_brand_device_os.value_counts()

device_brand  device_os
(not set)     Android      78165
              Tizen           99
              (not set)       30
              iOS             27
dtype: int64

In [45]:
df_brand = device_os_nan.device_brand.value_counts()
df_brand.head(50)

Apple         2972723
Samsung       1327178
Xiaomi        1056248
Huawei         723386
Realme          61264
OnePlus         60155
(not set)       54454
OPPO            43401
Vivo            41341
Asus            34500
Nokia           27909
Google          27491
Sony            27112
ZTE             16191
Meizu           14033
Motorola        10804
Lenovo          10140
LG               8178
Blackview        8070
Tecno            5646
BQ               3587
HTC              3039
Mozilla          2985
DOOGEE           2485
BlackBerry       2364
Umidigi          2324
Oukitel          2273
Alcatel          2116
itel             1785
Ulefone          1684
LeEco            1271
Vsmart            966
Sharp             845
Cubot             800
POCO              770
Philips           603
Infinix           589
TP-Link           561
Leagoo            485
Vertex            463
Neffos            452
Prestigio         415
Kingplay          388
DEXP              387
CAT               361
Wileyfox  

### Удаляем неинформативные столбцы, заполняем Nan значения

In [26]:
df_merged_ready = df_merged_ready.drop(columns=['device_model'], axis = 1) #удаляем столбцы где данные не заполнены свыше 60%
df_merged_ready

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target
0,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,360x780,Chrome,Russia,Saint Petersburg,0
1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,360x780,Chrome,Russia,Saint Petersburg,0
2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,360x780,Chrome,Russia,Saint Petersburg,0
3,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,360x780,Chrome,Russia,Saint Petersburg,0
4,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,wvtWRwiRmvPIsSQuXnvd,mobile,Android,Huawei,360x780,Chrome,Russia,Saint Petersburg,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15685214,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,428x926,Safari,Russia,Moscow,0
15685215,IZEXUFLARCUMynmHNBGo,blogger_channel,foFTSdUvNqqkPzZvgiqt,DZYjhfIUfdqhfuTNUmjn,puhZPIYqKXeFPaUviSjo,mobile,Android,Realme,360x800,Chrome,Russia,Ulyanovsk,0
15685216,QxAxdyPLuQMEcrdZWdWb,cpa,mSVMBvlHFgftJaQAZEdp,NaN,NaN,mobile,iOS,Apple,375x667,Safari (in-app),Russia,Samara,0
15685217,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,tVtbIKrPSOvrXLCznVVe,desktop,Windows,NaN,1366x768,Edge,Russia,Orenburg,0


In [18]:
print(((df_merged_ready.isna().sum() / len(df_merged_ready)) * 100).sort_values(ascending=False)) #выводим какие столбцы остались и какой процент пропусков в них


utm_keyword                 58.684721
device_os                   58.388614
device_brand                25.156646
utm_adcontent               18.057880
utm_campaign                14.018759
utm_source                   0.004463
utm_medium                   0.000000
device_category              0.000000
device_screen_resolution     0.000000
device_browser               0.000000
geo_country                  0.000000
geo_city                     0.000000
target                       0.000000
dtype: float64


In [27]:
df_merged_ready = df_merged_ready.dropna()
df_merged_ready

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,geo_city,target
38,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Chelyabinsk,0
39,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Chelyabinsk,0
40,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Chelyabinsk,0
41,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Chelyabinsk,0
42,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,Chelyabinsk,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15685207,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),412x823,Chrome,Russia,Samara,0
15685210,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,Moscow,0
15685213,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,Chelyabinsk,0
15685214,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,iOS,Apple,428x926,Safari,Russia,Moscow,0


### Очистка данных

In [37]:
geo_city_unique = df_merged_ready.geo_city.value_counts()
geo_city_unique

Moscow              1454852
Saint Petersburg     542961
Yekaterinburg        154229
Krasnodar            148720
(not set)            132980
                     ...   
Sofiivka                  1
92323                     1
Auckland                  1
Palm Valley               1
Pa Tong                   1
Name: geo_city, Length: 1296, dtype: int64

In [72]:
df_merged_ready = df_merged_ready.set_index('geo_city')

In [73]:
df_merged_ready = df_merged_ready.drop(['9954', '86039', '8230', '8161', '7668', '74565', '7084', '6784', '61925', '59139', '13403', '24130', '53425', '75709', '83709', '92323', '9992'], axis=0)

In [78]:
df_merged_ready = df_merged_ready[df_merged_ready['utm_source'] != '(not set)']
df_merged_ready

,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target
geo_city,,,,,,,,,,,,
Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
...,...,...,...,...,...,...,...,...,...,...,...,...
Samara,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,(not set),412x823,Chrome,Russia,0
Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0
Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0


In [81]:
df_merged_ready = df_merged_ready[df_merged_ready['utm_medium'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['utm_campaign'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['utm_adcontent'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['utm_keyword'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['device_category'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['device_os'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['device_brand'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['device_screen_resolution'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['device_browser'] != '(not set)']
df_merged_ready = df_merged_ready[df_merged_ready['geo_country'] != '(not set)']

In [85]:
df_merged_ready = df_merged_ready[df_merged_ready['utm_medium'] != '(none)']

In [88]:
df_merged_ready.to_csv('~/downloads/df_merged_clean')

### Создаём новые фичи

In [128]:
df_clean = pd.read_csv('~/downloads/df_merged_clean')

In [129]:
df_clean = df_clean[df_clean['geo_city'] != '(not set)']
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0


In [93]:
df_clean_name_columns = df_clean.columns.values.tolist() #выводим все названия столбцов df
df_clean_name_columns

['geo_city',
 'utm_source',
 'utm_medium',
 'utm_campaign',
 'utm_adcontent',
 'utm_keyword',
 'device_category',
 'device_os',
 'device_brand',
 'device_screen_resolution',
 'device_browser',
 'geo_country',
 'target']

In [94]:
for x in df_clean_name_columns: #выводим все уникальные значения по столбцам
    print(f"Характеристика: {x} \nКоличество уникальных значений: {df_clean[x].nunique()} \nСписок значений: \n{df_clean[x].value_counts()} \n")

Характеристика: geo_city 
Количество уникальных значений: 1196 
Список значений: 
Moscow              1227763
Saint Petersburg     462388
Yekaterinburg        140961
Krasnodar            137091
Kazan                 94616
                     ...   
Lucan                     1
Ventspils                 1
Saint Michael             1
Weston                    1
Pa Tong                   1
Name: geo_city, Length: 1196, dtype: int64 

Характеристика: utm_source 
Количество уникальных значений: 112 
Список значений: 
ZpYIoDJMcFzVoPFsHGJL    3010935
MvfHsxITijuriZxsqZqt     158614
bByPQxmDaMXgpHeypKSM     109273
kjsLglQLzykiRbcDiGcD      95408
RmEBuqrriAfAVsLQQmhk      76273
                         ...   
FNIjOzNNxWDHGSGhdwpo          1
eGPqcrOyAyMvXPpbVMLD          1
fbFKcMumlScApQMqFIqp          1
cdPAGAvKeBGoqAkQBuGR          1
maiZOsuEAMdeoRVsYoFk          1
Name: utm_source, Length: 112, dtype: int64 

Характеристика: utm_medium 
Количество уникальных значений: 33 
Список значений: 
ba

In [130]:
df_clean['russia_or_not'] = df_clean.apply(lambda x: 1 if x.geo_country == "Russia" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1


In [96]:
target_utm_keyword = df_clean.groupby('target')['utm_keyword'].agg(pd.Series.mode).reset_index()
target_utm_keyword

,target,utm_keyword
0,0,puhZPIYqKXeFPaUviSjo
1,1,puhZPIYqKXeFPaUviSjo


Проверить какое популярное значение для таргета 1 и добавить новые фичи исходя из этого. Декодировать бранд модель и бренд ос

In [131]:
df_clean['utm_keyword_0_1'] = df_clean.apply(lambda x: 0 if x.utm_keyword == "puhZPIYqKXeFPaUviSjo" else 1, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not,utm_keyword_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1,0


In [99]:
target_hit_referer = df_clean.groupby('target')['utm_source'].agg(pd.Series.mode).reset_index()
target_hit_referer

,target,utm_source
0,0,ZpYIoDJMcFzVoPFsHGJL
1,1,ZpYIoDJMcFzVoPFsHGJL


In [132]:
df_clean['utm_source_0_1'] = df_clean.apply(lambda x: 1 if x.utm_source == "ZpYIoDJMcFzVoPFsHGJL" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not,utm_keyword_0_1,utm_source_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1,1,0
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1,1,0
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1,1,0
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1,0,1


In [101]:
target_event_label = df_clean.groupby('target')['utm_medium'].agg(pd.Series.mode).reset_index()
target_event_label

,target,utm_medium
0,0,banner
1,1,banner


In [133]:
df_clean['utm_medium_0_1'] = df_clean.apply(lambda x: 0 if x.utm_medium == "banner" else 1, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1,1,0,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1,1,0,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1,1,0,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1,0,1,0


In [109]:
df_geo_city_target = df_clean.groupby('geo_city')['target'].count().reset_index()
df_geo_city_target

,geo_city,target
0,10th of Ramadan City,9
1,Abakan,2668
2,Abu Dhabi,9
3,Abuja,15
4,Accra,1
...,...,...
1191,Zvenigorod,1307
1192,Zwickau,2
1193,Тарасовка,14
1194,Тимофеевка,68


In [112]:
target_utm_campaign = df_clean.groupby('target')['utm_campaign'].agg(pd.Series.mode).reset_index()
target_utm_campaign

,target,utm_campaign
0,0,LEoPHuyFvzoNfnzGgfcd
1,1,LEoPHuyFvzoNfnzGgfcd


In [134]:
df_clean['utm_campaign_0_1'] = df_clean.apply(lambda x: 0 if x.utm_campaign == "LEoPHuyFvzoNfnzGgfcd" else 1, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1,1,0,1,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1,1,0,1,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1,1,0,1,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1,0,1,0,0


In [114]:
target_utm_adcontent = df_clean.groupby('target')['utm_adcontent'].agg(pd.Series.mode).reset_index()
target_utm_adcontent

,target,utm_adcontent
0,0,vCIpmpaGBnIQhyYNkXqp
1,1,JNHcPlZPxEMWDnRiyoBf


In [135]:
df_clean['utm_adcontent_0_1'] = df_clean.apply(lambda x: 1 if x.utm_adcontent == "JNHcPlZPxEMWDnRiyoBf" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1,1,0,1,1,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1,1,0,1,1,0
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1,1,0,1,1,0
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1,0,1,0,0,0


In [116]:
target_device_os = df_clean.groupby('target')['device_os'].agg(pd.Series.mode).reset_index()
target_device_os

,target,device_os
0,0,Android
1,1,Android


In [136]:
df_clean['device_os_0_1'] = df_clean.apply(lambda x: 1 if x.device_os == "Android" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,device_browser,geo_country,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,Chrome,Russia,0,1,0,1,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,Chrome,Russia,0,1,1,0,1,1,1,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,Chrome,Russia,1,1,1,0,1,1,0,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,Chrome,Russia,0,1,1,0,1,1,0,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,Chrome,Russia,0,1,0,1,0,0,0,1


In [118]:
target_device_brand = df_clean.groupby('target')['device_brand'].agg(pd.Series.mode).reset_index()
target_device_brand

,target,device_brand
0,0,Samsung
1,1,Samsung


In [137]:
df_clean['device_brand_0_1'] = df_clean.apply(lambda x: 1 if x.device_brand == "Samsung" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,...,geo_country,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,Russia,0,1,0,1,0,1,1,1,0
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,Russia,0,1,0,1,0,1,1,1,0
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,Russia,0,1,0,1,0,1,1,1,0
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,Russia,0,1,0,1,0,1,1,1,0
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,Russia,0,1,0,1,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,...,Russia,0,1,1,0,1,1,1,1,0
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,...,Russia,1,1,1,0,1,1,0,1,0
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,...,Russia,0,1,1,0,1,1,0,1,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,...,Russia,0,1,0,1,0,0,0,1,1


In [121]:
target_device_screen_resolution = df_clean.groupby('target')['device_screen_resolution'].agg(pd.Series.mode).reset_index()
target_device_screen_resolution

,target,device_screen_resolution
0,0,393x851
1,1,393x851


In [138]:
df_clean['device_screen_resolution_0_1'] = df_clean.apply(lambda x: 0 if x.device_screen_resolution == "393x851" else 1, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,...,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,0,1,1,1,0,0
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,0,1,1,1,0,0
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,0,1,1,1,0,0
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,0,1,1,1,0,0
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,...,0,1,1,0,1,1,1,1,0,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,...,1,1,1,0,1,1,0,1,0,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,...,0,1,1,0,1,1,0,1,1,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,...,0,1,0,1,0,0,0,1,1,1


In [123]:
target_device_category = df_clean.groupby('target')['device_category'].agg(pd.Series.mode).reset_index()
target_device_category

,target,device_category
0,0,mobile
1,1,mobile


In [139]:
df_clean['device_category_0_1'] = df_clean.apply(lambda x: 1 if x.device_category == "mobile" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,...,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,device_category_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,1,0,1,0,1,1,1,0,0,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,1,0,1,0,1,1,1,0,0,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,1,0,1,0,1,1,1,0,0,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,1,0,1,0,1,1,1,0,0,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,1,0,1,0,1,1,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,...,1,1,0,1,1,1,1,0,1,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,...,1,1,0,1,1,0,1,0,1,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,...,1,1,0,1,1,0,1,1,1,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,...,1,0,1,0,0,0,1,1,1,1


In [125]:
target_device_browser = df_clean.groupby('target')['device_browser'].agg(pd.Series.mode).reset_index()
target_device_browser

,target,device_browser
0,0,Chrome
1,1,Chrome


In [140]:
df_clean['device_browser_0_1'] = df_clean.apply(lambda x: 1 if x.device_browser == "Chrome" else 0, axis=1)
df_clean

,geo_city,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_resolution,...,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,device_category_0_1,device_browser_0_1
0,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,1,1,0,0,1,1
1,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,1,1,0,0,1,1
2,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,1,1,0,0,1,1
3,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,1,1,0,0,1,1
4,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,zxoiLxhuSIFrCeTLQVWZ,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Xiaomi,393x851,...,0,1,0,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897059,Moscow,kjsLglQLzykiRbcDiGcD,cpc,XHNUiSKKGTEpSAlaVMFQ,JNHcPlZPxEMWDnRiyoBf,hAmNSZmQkKQKAjZEGlgb,mobile,Android,Xiaomi,360x640,...,1,0,1,1,1,1,0,1,1,1
3897060,Moscow,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Huawei,360x771,...,1,0,1,1,0,1,0,1,1,1
3897061,Moscow,vFcAhRxLfOWKhvxjELkx,organic,okTXSMadDkjvntEHzIjp,LLfCasrxQzJIyuldcuWy,aXQzDWsJuGXeBXexNHjc,mobile,Android,Samsung,412x915,...,1,0,1,1,0,1,1,1,1,1
3897062,Chelyabinsk,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Samsung,412x869,...,0,1,0,0,0,1,1,1,1,1


In [141]:
df_clean.corr()

,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,device_category_0_1,device_browser_0_1
target,1.000000,0.001617,0.004947,0.000327,0.004080,0.004558,0.007166,0.003914,0.007206,0.000744,0.000839,0.003994
russia_or_not,0.001617,1.000000,0.013919,-0.003377,-0.004617,-0.001336,-0.009645,0.018032,-0.002565,0.003115,0.011508,0.012504
utm_keyword_0_1,0.004947,0.013919,1.000000,-0.652725,0.540054,0.378738,-0.123879,0.062910,-0.005416,0.004751,-0.041481,-0.029058
utm_source_0_1,0.000327,-0.003377,-0.652725,1.000000,-0.808089,-0.574356,-0.100613,0.037942,0.044981,-0.024140,0.056131,0.155500
utm_medium_0_1,0.004080,-0.004617,0.540054,-0.808089,1.000000,0.695970,0.209707,-0.043850,-0.033075,0.017521,-0.041947,-0.138832
utm_campaign_0_1,0.004558,-0.001336,0.378738,-0.574356,0.695970,1.000000,0.553691,-0.055487,-0.037624,0.020052,-0.031045,-0.121930
utm_adcontent_0_1,0.007166,-0.009645,-0.123879,-0.100613,0.209707,0.553691,1.000000,-0.071215,-0.015772,0.019176,-0.011395,-0.060265
device_os_0_1,0.003914,0.018032,0.062910,0.037942,-0.043850,-0.055487,-0.071215,1.000000,0.379153,-0.198095,-0.012221,0.857255
device_brand_0_1,0.007206,-0.002565,-0.005416,0.044981,-0.033075,-0.037624,-0.015772,0.379153,1.000000,0.198885,-0.017588,0.290997
device_screen_resolution_0_1,0.000744,0.003115,0.004751,-0.024140,0.017521,0.020052,0.019176,-0.198095,0.198885,1.000000,-0.029821,-0.201741


In [142]:
df_clean.to_csv('~/downloads/df_clean', index=False)

## Кодируем фичи

In [2]:
df_clean = pd.read_csv('~/downloads/df_clean')

In [3]:
# Удаляем ненужные/лишние колонки 

columns_to_drop = [
    'utm_source',
    'utm_campaign',
    'utm_adcontent',
    'utm_keyword',
    'device_brand',
    'device_screen_resolution',
    'geo_country'
]

df_clean.drop(columns_to_drop, axis=1, inplace=True) 

In [145]:
df_clean

,geo_city,utm_medium,device_category,device_os,device_browser,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,device_category_0_1,device_browser_0_1
0,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1
1,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1
2,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1
3,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1
4,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3781334,Moscow,cpc,mobile,Android,Chrome,0,1,1,0,1,1,1,1,0,1,1,1
3781335,Moscow,cpm,mobile,Android,Chrome,1,1,1,0,1,1,0,1,0,1,1,1
3781336,Moscow,organic,mobile,Android,Chrome,0,1,1,0,1,1,0,1,1,1,1,1
3781337,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,0,0,1,1,1,1,1


In [4]:
df_geo_city_target = df_clean.groupby('geo_city')['target'].count().reset_index()
df_geo_city_target

,geo_city,target
0,10th of Ramadan City,9
1,Abakan,2668
2,Abu Dhabi,9
3,Abuja,15
4,Accra,1
...,...,...
1191,Zvenigorod,1307
1192,Zwickau,2
1193,Тарасовка,14
1194,Тимофеевка,68


In [5]:
df_target_20000 = df_geo_city_target[df_geo_city_target['target'] > 6000]
df_target_20000.head(50)

,geo_city,target
67,Astrakhan,7807
82,Balashikha,17871
92,Barnaul,10116
105,Belgorod,7459
194,Cheboksary,6779
197,Chelyabinsk,69093
201,Cherkessk,7144
209,Chita,12850
249,Domodedovo,6034
338,Grozny,62390


In [6]:
df_target_20000.rename(columns = {'target':'target_count'}, inplace = True)
df_target_20000

C:\Users\Полина\AppData\Local\Temp\ipykernel_9132\3436290996.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_target_20000.rename(columns = {'target':'target_count'}, inplace = True)


,geo_city,target_count
67,Astrakhan,7807
82,Balashikha,17871
92,Barnaul,10116
105,Belgorod,7459
194,Cheboksary,6779
197,Chelyabinsk,69093
201,Cherkessk,7144
209,Chita,12850
249,Domodedovo,6034
338,Grozny,62390


In [7]:
df_merged_target = pd.merge(
    df_clean, df_target_20000,
    left_on='geo_city',
    right_on='geo_city',
    how='inner'
) #объединям два датафрейма и запысываем в переменную

df_merged_target

,geo_city,utm_medium,device_category,device_os,device_browser,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,device_category_0_1,device_browser_0_1,target_count
0,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1,69093
1,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1,69093
2,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1,69093
3,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1,69093
4,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,1,1,1,0,0,1,1,69093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3369067,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,0,0,1,0,1,1,1,6590
3369068,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,0,0,1,0,1,1,1,6590
3369069,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,0,1,1,0,1,1,1,6590
3369070,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,0,0,1,0,1,1,1,6590


In [8]:
ohe = OneHotEncoder(sparse=False)
ohe

OneHotEncoder(sparse=False)

In [9]:
ohe.fit(df_merged_target[['geo_city']])

OneHotEncoder(sparse=False)

In [10]:
ohe_fuel = ohe.transform(df_merged_target[['geo_city']])
ohe_fuel

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
ohe.get_feature_names()

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_Astrakhan', 'x0_Balashikha', 'x0_Barnaul', 'x0_Belgorod',
       'x0_Cheboksary', 'x0_Chelyabinsk', 'x0_Cherkessk', 'x0_Chita',
       'x0_Domodedovo', 'x0_Grozny', 'x0_Irkutsk', 'x0_Ivanovo',
       'x0_Izhevsk', 'x0_Kaliningrad', 'x0_Kaluga', 'x0_Kazan',
       'x0_Kemerovo', 'x0_Khabarovsk', 'x0_Khimki', 'x0_Kirov',
       'x0_Krasnodar', 'x0_Krasnoyarsk', 'x0_Kursk', 'x0_Lipetsk',
       'x0_Makhachkala', 'x0_Moscow', 'x0_Naberezhnye Chelny',
       'x0_Nizhny Novgorod', 'x0_Novorossiysk', 'x0_Novosibirsk',
       'x0_Odintsovo', 'x0_Omsk', 'x0_Orenburg', 'x0_Penza', 'x0_Perm',
       'x0_Pyatigorsk', 'x0_Rostov-on-Don', 'x0_Ryazan',
       'x0_Saint Petersburg', 'x0_Samara', 'x0_Saratov', 'x0_Smolensk',
       'x0_Sochi', 'x0_Stavropol', 'x0_Surgut', 'x0_Tomsk', 'x0_Tula',
       'x0_Tver', 'x0_Tyumen', 'x0_Ufa', 'x0_Ulan-Ude', 'x0_Ulyanovsk',
       'x0_Vidnoye', 'x0_Vladimir', 'x0_Vladivostok', 'x0_Volgograd',
       'x0_Voronezh', 'x0_Yaroslavl', 'x0_Yekaterinburg',


In [11]:
df_merged_target[ohe.get_feature_names()] = ohe_fuel

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [12]:
ohe.fit(df_merged_target[['utm_medium']])
ohe_fuel = ohe.transform(df_merged_target[['utm_medium']])
ohe.get_feature_names()
df_merged_target[ohe.get_feature_names()] = ohe_fuel

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [13]:
ohe.fit(df_merged_target[['device_category']])
ohe_fuel = ohe.transform(df_merged_target[['device_category']])
ohe.get_feature_names()
df_merged_target[ohe.get_feature_names()] = ohe_fuel

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
ohe.fit(df_merged_target[['device_os']])
ohe_fuel = ohe.transform(df_merged_target[['device_os']])
ohe.get_feature_names()
df_merged_target[ohe.get_feature_names()] = ohe_fuel

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [15]:
ohe.fit(df_merged_target[['device_browser']])
ohe_fuel = ohe.transform(df_merged_target[['device_browser']])
ohe.get_feature_names()
df_merged_target[ohe.get_feature_names()] = ohe_fuel

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
ohe.get_feature_names()

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['x0_Android Browser', 'x0_Android Webview', 'x0_Chrome',
       'x0_Coc Coc', 'x0_Firefox', 'x0_Iron', 'x0_Maxthon', 'x0_Opera',
       'x0_Puffin', 'x0_Safari', 'x0_Safari (in-app)',
       'x0_Samsung Internet', 'x0_UC Browser', 'x0_YaBrowser',
       'x0_com.zhiliaoapp.musically'], dtype=object)

In [40]:
df_merged_target

,geo_city,utm_medium,device_category,device_os,device_browser,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,...,x0_Iron,x0_Maxthon,x0_Opera,x0_Puffin,x0_Safari,x0_Safari (in-app),x0_Samsung Internet,x0_UC Browser,x0_YaBrowser,x0_com.zhiliaoapp.musically
0,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Chelyabinsk,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3369067,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369068,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369069,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369070,Kirov,banner,mobile,Android,Chrome,0,1,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# Удаляем ненужные/лишние колонки 

columns_to_drop = [
    'geo_city',
    'utm_medium',
    'device_category',
    'device_os',
    'device_browser'
]

df_merged_target.drop(columns_to_drop, axis=1, inplace=True) 

In [42]:
df_merged_target

,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,...,x0_Iron,x0_Maxthon,x0_Opera,x0_Puffin,x0_Safari,x0_Safari (in-app),x0_Samsung Internet,x0_UC Browser,x0_YaBrowser,x0_com.zhiliaoapp.musically
0,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3369067,0,1,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369068,0,1,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369069,0,1,0,1,0,0,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369070,0,1,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_merged_target.corr()

,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,...,x0_Iron,x0_Maxthon,x0_Opera,x0_Puffin,x0_Safari,x0_Safari (in-app),x0_Samsung Internet,x0_UC Browser,x0_YaBrowser,x0_com.zhiliaoapp.musically
target,1.000000,NaN,0.004465,0.001037,0.003492,0.004076,0.007089,0.004859,0.007670,0.000617,...,-5.568549e-05,-0.000162,-0.002990,-0.000281,-0.004838,-0.001372,0.003882,-0.001482,-0.002664,-0.000039
russia_or_not,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
utm_keyword_0_1,0.004465,NaN,1.000000,-0.654510,0.536738,0.378799,-0.130772,0.066255,-0.005460,0.002883,...,-2.578638e-04,-0.000752,0.039932,0.004781,-0.090855,0.051876,0.119346,0.029079,0.077285,0.001628
utm_source_0_1,0.001037,NaN,-0.654510,1.000000,-0.800634,-0.572780,-0.092830,0.033352,0.044637,-0.023119,...,-1.506765e-03,-0.004393,-0.066372,-0.002527,0.028299,-0.154012,-0.139283,-0.025561,-0.172357,-0.001065
utm_medium_0_1,0.003492,NaN,0.536738,-0.800634,1.000000,0.700350,0.209707,-0.037431,-0.031612,0.016242,...,1.235640e-03,0.003014,0.054261,0.001652,-0.012282,0.125242,0.114835,0.019726,0.136596,0.000874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
x0_Safari (in-app),-0.001372,NaN,0.051876,-0.154012,0.125242,0.088935,0.030821,-0.126645,-0.048429,0.025463,...,-6.103316e-05,-0.000178,-0.004215,-0.000308,-0.047795,1.000000,-0.010752,-0.001624,-0.014141,-0.000043
x0_Samsung Internet,0.003882,NaN,0.119346,-0.139283,0.114835,0.080164,-0.006288,0.084897,0.220410,0.043408,...,-1.045765e-04,-0.000305,-0.007223,-0.000528,-0.081893,-0.010752,1.000000,-0.002783,-0.024229,-0.000074
x0_UC Browser,-0.001482,NaN,0.029079,-0.025561,0.019726,0.018772,-0.003341,0.012439,0.002302,0.003559,...,-1.579893e-05,-0.000046,-0.001091,-0.000080,-0.012372,-0.001624,-0.002783,1.000000,-0.003660,-0.000011
x0_YaBrowser,-0.002664,NaN,0.077285,-0.172357,0.136596,0.099582,0.013916,0.065337,-0.015599,0.024149,...,-1.375374e-04,-0.000401,-0.009500,-0.000695,-0.107705,-0.014141,-0.024229,-0.003660,1.000000,-0.000097


In [44]:
df_merged_target.to_csv('~/downloads/df_clean_prepared', index=False)

# Modeling

In [2]:
df_prepared = pd.read_csv('~/downloads/df_clean_prepared')

In [3]:
df_prepared

,target,russia_or_not,utm_keyword_0_1,utm_source_0_1,utm_medium_0_1,utm_campaign_0_1,utm_adcontent_0_1,device_os_0_1,device_brand_0_1,device_screen_resolution_0_1,...,x0_Iron,x0_Maxthon,x0_Opera,x0_Puffin,x0_Safari,x0_Safari (in-app),x0_Samsung Internet,x0_UC Browser,x0_YaBrowser,x0_com.zhiliaoapp.musically
0,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,1,0,1,0,1,1,1,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3369067,0,1,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369068,0,1,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369069,0,1,0,1,0,0,1,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3369070,0,1,0,1,0,0,0,1,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
missing_values_df_merged_1 = ((df_prepared.isna().sum() / len(df_prepared)) * 100).sort_values(ascending=False) #формула которая отоброжает процент пропущенных значений

print('Процент пропущенных значений:')
missing_values_df_merged_1

Процент пропущенных значений:


target                         0.0
russia_or_not                  0.0
x0_partner                     0.0
x0_outlook                     0.0
x0_organic                     0.0
                              ... 
x0_Makhachkala                 0.0
x0_Lipetsk                     0.0
x0_Kursk                       0.0
x0_Krasnoyarsk                 0.0
x0_com.zhiliaoapp.musically    0.0
Length: 128, dtype: float64

In [11]:
df_prepared = df_prepared.astype({'x0_Balashikha': np.int32, 'x0_Barnaul': np.int32, 'x0_Belgorod': np.int32, 'x0_Cheboksary': np.int32,
       'x0_Chelyabinsk': np.int32, 'x0_Chita': np.int32, 'x0_Dolgoprudny': np.int32, 'x0_Domodedovo': np.int32,
       'x0_Grozny': np.int32, 'x0_Irkutsk': np.int32, 'x0_Ivanovo': np.int32, 'x0_Izhevsk': np.int32,
       'x0_Kaliningrad': np.int32, 'x0_Kaluga': np.int32, 'x0_Kazan': np.int32, 'x0_Kemerovo': np.int32,
       'x0_Khabarovsk': np.int32, 'x0_Khimki': np.int32, 'x0_Kirov': np.int32, 'x0_Korolyov': np.int32,
       'x0_Krasnodar': np.int32, 'x0_Krasnogorsk': np.int32, 'x0_Krasnoyarsk': np.int32, 'x0_Lipetsk': np.int32,
       'x0_Lyubertsy': np.int32, 'x0_Makhachkala': np.int32, 'x0_Moscow': np.int32, 'x0_Mytishchi': np.int32,
       'x0_Naberezhnye Chelny': np.int32, 'x0_Nizhny Novgorod': np.int32, 'x0_Novosibirsk': np.int32,
       'x0_Odintsovo': np.int32, 'x0_Omsk': np.int32, 'x0_Orenburg': np.int32, 'x0_Perm': np.int32, 'x0_Podolsk': np.int32,
       'x0_Pushkino': np.int32, 'x0_Pyatigorsk': np.int32, 'x0_Reutov': np.int32, 'x0_Rostov-on-Don': np.int32,
       'x0_Ryazan': np.int32, 'x0_Saint Petersburg': np.int32, 'x0_Samara': np.int32, 'x0_Saratov': np.int32,
       'x0_Smolensk': np.int32, 'x0_Sochi': np.int32, 'x0_Stavropol': np.int32, 'x0_Surgut': np.int32,
       'x0_Tolyatti': np.int32, 'x0_Tomsk': np.int32, 'x0_Tula': np.int32, 'x0_Tver': np.int32, 'x0_Tyumen': np.int32,
       'x0_Ufa': np.int32, 'x0_Ulyanovsk': np.int32, 'x0_Vidnoye': np.int32, 'x0_Vladimir': np.int32,
       'x0_Vladivostok': np.int32, 'x0_Volgograd': np.int32, 'x0_Voronezh': np.int32, 'x0_Yaroslavl': np.int32,
       'x0_Yekaterinburg': np.int32, 'x0_Zheleznodorozhny': np.int32, })

In [5]:
df_prepared.to_csv('~/downloads/df_clean_prepared_ohe', index=False)

## Logistic Regression

In [2]:
df_prepared = pd.read_csv('~/downloads/df_clean_prepared_ohe')

In [3]:
df_modeling1 = df_modeling.iloc[:1414696]
df_modeling2 = df_modeling.iloc[1414696:2829392]
df_modeling3 = df_modeling.iloc[2829392:4244088]
df_modeling4 = df_modeling.iloc[4244088:5658784]

In [3]:
x = df_prepared.drop(['target'], axis=1) #делим датасет на целевую перменную и переменные обучения
y = df_prepared.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42) #разделяем данные на тренировочную и тестовую выборки 70 на 30

In [9]:
best_logreg = LogisticRegression(penalty='l1', solver='saga', random_state=42) #задаём параметры модели и инициализируем переменную

best_logreg.fit(x_train, y_train) #обучаем модель

logreg_pred = best_logreg.predict(x_test) #предсказываем переменную на новых данных 
accuracy_score(y_test, logreg_pred) #смотрим какая ошибка 

D:\Anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.9948650568603434

## Random Forest

In [6]:
rf_clf = RandomForestClassifier(random_state=42) #помещаем модель случайного леса в переменную
rf_clf.fit(x_train, y_train)

RandomForestClassifier(random_state=42)

In [7]:
predicted_train_rf = rf_clf.predict(x_train) #обучаем модель случайного леса на тренировочных данных
predicted_test_rf = rf_clf.predict(x_test) #обучаем модель случайного леса на тестовой выборке

print(accuracy_score(y_train, predicted_train_rf)) #проверяем как обучилась модель на тренировочных данных
print(accuracy_score(y_test, predicted_test_rf)) #проверяем как модель предсказывает целевую метрику на новых данных

0.9947806729281066
0.9948640674686017


## MLPClassifier

In [5]:
mlp = MLPClassifier(random_state=42, max_iter=500, hidden_layer_sizes=(100,60,40,20)) #инициализируем модель нейронной сети для задач классификации
mlp.fit(x_train, y_train) #обучаем нейронную сеть на тренировочной выборке

MLPClassifier(hidden_layer_sizes=(100, 60, 40, 20), max_iter=500,
              random_state=42)

In [9]:
pred_mlp = mlp.predict(x_test) #предсказываем данные с помощью обученной нейросети
accuracy_score(y_test, pred_mlp) #проверяем точность нейросети

0.9948650568603434

## Кросс-валидация моделей

In [8]:
logreg_cross = cross_val_score(best_logreg, x, y, cv=4) #проводим кросс валидацию модели логистической регрессии
logreg_cross #выводим результат

NameError: name 'best_logreg' is not defined

In [11]:
rf_cross = cross_val_score(rf_clf, x, y, cv=4) #проводим кросс валидацию модели случайного леса и выводим результат
rf_cross

array([0.9948045 , 0.91197101, 0.92141219, 0.99480332])

In [6]:
mlp_cross = cross_val_score(mlp, x, y, cv=4) #проводим кросс валидацию модели многослойного перцептрона и выводим результат
mlp_cross

array([0.9948045 , 0.99480332, 0.99480332, 0.99480332])

In [ ]:
print(sum(logreg_cross), sum(rf_cross), sum(mlp_cross)) #выясняем, какая модель показала лучшие результаты на кросс-валидации
print(f'Лучшая модель: Random Forest')

In [ ]:
df1 = pd.read_csv('skillbox_diploma_main_dataset_sberautopodpiska-20230601T122423Z-003/skillbox_diploma_main_dataset_sberautopodpiska/ga_sessions.csv')
df1.head(10)

In [33]:
x = df_prepared.drop(['target'], axis=1)
y = df_prepared.target

best_predicted_test_rf = rf_clf.predict(x_test) 

print(accuracy_score(y_test, best_predicted_test_rf)) 

#проверяем модель на тестовых данных

0.9965669908036823


## Results

In [7]:
filename = 'sber_mlp_model.pickle' #сериализуем лучшую модель в файл огурчика

with open(filename, 'wb') as file:

    pickle.dump(mlp, file) # logreg — наша модель

NameError: name 'best_logreg' is not defined